In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [128]:
# Web scraping using BeautifulSoup
wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(wiki_link)
page_content = BeautifulSoup(source.content, "html.parser")
table = page_content.find_all('table')[0] 
df = pd.read_html(str(table))[0]

# Renaming columns
df.columns = ('Postcode','Borough','Neighbourhood')
df = df[1:]

# Removing where Borough = 'Not assigned'
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True,inplace=True)

# # Combining neighbourhoods for similar postcode
df = df.groupby(by=['Postcode','Borough']).apply(lambda x: "%s" % ', '.join(x['Neighbourhood']))
# .to_frame()
# df.columns('Postcode','Borough','Neighbourhoods')

In [129]:
list(df)
df

Postcode  Borough         
M1B       Scarborough                                            Rouge, Malvern
M1C       Scarborough                    Highland Creek, Rouge Hill, Port Union
M1E       Scarborough                         Guildwood, Morningside, West Hill
M1G       Scarborough                                                    Woburn
M1H       Scarborough                                                 Cedarbrae
M1J       Scarborough                                       Scarborough Village
M1K       Scarborough               East Birchmount Park, Ionview, Kennedy Park
M1L       Scarborough                           Clairlea, Golden Mile, Oakridge
M1M       Scarborough           Cliffcrest, Cliffside, Scarborough Village West
M1N       Scarborough                               Birch Cliff, Cliffside West
M1P       Scarborough         Dorset Park, Scarborough Town Centre, Wexford ...
M1R       Scarborough                                         Maryvale, Wexford
M1S       Sca